In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

from __future__ import division
import os
import sys
import skimage
import numpy as np
from scipy import spatial
from sklearn import svm
from logging.config import fileConfig
import time


sys.path.append("SVMMatching/")
from facetool import FaceTool


sys.path.append("FeatureExtraction/")

#Add caffe path
sys.path.append("/home/iclab/ibm/face-recognition/caffe/caffe/python")
import caffe
import cv2
from VGGFeatureExtractor import VGGExtractor


from sklearn.externals import joblib

In [2]:
classifier = joblib.load('svm.pkl') 

target_img_rows = 224
target_img_cols = 224

#----Test One Face Picture-----#
vgg_face_caffe_path =  'vgg_face_caffe/'
model_file          = vgg_face_caffe_path + '/VGG_FACE_deploy.prototxt'
pretrained_file     = vgg_face_caffe_path + '/VGG_FACE.caffemodel'
vggExtractor = VGGExtractor(model_file,pretrained_file)

ftool = FaceTool('./')

In [3]:
#--------------Test One Face---------------#
def prdict_one_face(im_face):
    
    
    img = np.asarray(cv2.resize(im_face, (target_img_rows,target_img_cols)).astype(np.float32) )
    
    
    vggExtractor_time = time.time() 
    
    featureVector = vggExtractor.GetFeature(img)

    print("--- Use %s seconds for only vggExtractor ---" % ( time.time() - vggExtractor_time))

    
    featureVector = np.asarray(featureVector['fc7'])[0,:]
    
    
    vMax = max(featureVector)
    fea_div_vMax = featureVector/vMax
    fea_div_vMax_ary = []
    fea_div_vMax_ary.append(fea_div_vMax)
    
    
    
    before_predict_time = time.time()
    pre_result = classifier.predict(fea_div_vMax_ary)[0]
    print('pre_result',pre_result)
    
    print("--- Use %s seconds only classifier.predict---" % ( time.time() - before_predict_time))
    
    return pre_result
    
    # clf=svm.SVC(probability=True), you need to retrain it 
    p_p = classifier.predict_proba(fea_div_vMax_ary)
    p_p = np.squeeze(p_p)
#     print('p_p',p_p,'p_p.shape', p_p.shape)
    
    print('p_p[{0}] = {1}, np.argmax(p_p) = {2},arg_same={3}'.\
          format(pre_result, p_p[pre_result], np.argmax(p_p), (pre_result==np.argmax(p_p)) )   )
    
    return pre_result if p_p[pre_result] > 0.2 else -1


img_path = 'lfw_only6/Aaron_Sorkin/Aaron_Sorkin_0002.jpg'
im = cv2.imread(img_path)
face_ind = prdict_one_face(im)
print ftool.get_face_label(face_ind)

--- Use 0.0963129997253 seconds for only vggExtractor ---
('pre_result', 3)
--- Use 0.000669002532959 seconds only classifier.predict---
Aaron_Sorkin
